In [ ]:
import sys
sys.path.append('src')
import pandas as pd
import numpy as np
import seaborn as sns
from ploting import *
beatify(fig_width=6)
%matplotlib inline

In [ ]:
train_data = pd.read_csv('data/train_data_week_1_challenge.csv')
test_data = pd.read_csv('data/test_data_week_1_challenge.csv')

In [ ]:
train_data.columns

In [ ]:
train_data.head()

In [ ]:
len(train_data)

In [ ]:
train_data.shape

In [ ]:
train_data['continue_drop'].unique()

In [ ]:
train_data['caste'].unique()

In [ ]:
train_data['guardian'].unique()

In [ ]:
train_data['internet'].unique()

In [ ]:
train_data['gender'].unique()

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data["total_toilets"].fillna(train_data["total_toilets"].mean(), inplace=True)
train_data["establishment_year"].fillna(train_data["establishment_year"].mean(), inplace=True)

In [ ]:
copy_train_data = train_data
copy_train_data.drop('student_id', axis=1, inplace=True)
copy_train_data.drop('school_id', axis=1, inplace=True)

In [ ]:
copy_train_data.gender.replace(['F','M'], value=[0, 1], inplace=True)
copy_train_data.caste.replace(['BC','OC','SC','ST'], value=[0, 1, 2, 3,], inplace=True)
copy_train_data.guardian.replace(['father','mixed','mother','other'], value=[0, 1, 2, 3], inplace=True)
copy_train_data.continue_drop.replace(['continue','drop'], value=[1, 0], inplace=True)

In [ ]:
corr_list = copy_train_data.corr()['continue_drop'].sort_values(axis=0,ascending=False).iloc[1:]
corr_list

In [ ]:
beatify(fig_width=8)
sns.set(font_scale=1)  
sns.heatmap(copy_train_data.corr(),annot=True, fmt=".2f")

In [ ]:
feature = ['gender', 'caste', 'mathematics_marks',
       'english_marks', 'science_marks', 'science_teacher',
       'languages_teacher', 'guardian', 'internet',
          'total_students', 'total_toilets', 'establishment_year']

X = copy_train_data[feature]
y = copy_train_data.continue_drop

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Define a model
rforest = RandomForestRegressor()
# Fit the model
rforest.fit(X,y)

In [ ]:
beatify(fig_width=8)
imp_feat_rf = pd.Series(rforest.feature_importances_, index=X.columns).sort_values(ascending=False)
imp_feat_rf.plot(kind='bar', title='Feature Importance with Random Forest', color='C0')
plt.ylabel('Feature Importance values')

In [ ]:
copy_train_data.drop('total_toilets', axis=1, inplace=True)
copy_train_data.drop('establishment_year', axis=1, inplace=True)
copy_train_data.drop('total_students', axis=1, inplace=True)
copy_train_data.columns

In [ ]:
feature = ['gender', 'caste', 'mathematics_marks',
       'english_marks', 'science_marks', 'science_teacher',
       'languages_teacher', 'guardian', 'internet']
X = copy_train_data[feature]
y = copy_train_data.continue_drop

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 22)

In [ ]:
def regression_fit(X_train, y_train, X_test, y_test):
    
    rf = RandomForestRegressor()
    dt = DecisionTreeRegressor()
    knn =  KNeighborsClassifier()
    svm = SVC()
    
    models = { "RF":rf, "DT":dt, "SVM":svm, "KNN":knn}
    results = {}
    
    for (key,model) in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
#         print(y_pred)
        results[key]=f1_score(y_test, y_pred)
        
    return results   

In [ ]:
results = regression_fit(x_train, y_train, x_test, y_test)

In [ ]:
results

In [ ]:
dt = DecisionTreeRegressor()
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)
f1_jibu = f1_score(y_test, y_pred)
print('%.5f' % f1_jibu)
print(y_pred[0:10])
print(y_test[0:10])

In [ ]:
test_data.head()

In [ ]:
test_data['caste'].unique()

In [ ]:
selected_feature = ['gender', 'caste', 'mathematics_marks',
       'english_marks', 'science_marks', 'science_teacher',
       'languages_teacher', 'guardian', 'internet']
X_req_test = test_data[selected_feature]
X_req_test.gender.replace(['F','M'], value=[0, 1], inplace=True)
X_req_test.caste.replace(['BC','OC','SC','ST'], value=[0, 1, 2, 3,], inplace=True)
X_req_test.guardian.replace(['father','mixed','mother','other'], value=[0, 1, 2, 3], inplace=True)

X_req_test.columns

In [ ]:
y_outcome = dt.predict(X_req_test)
np.save("predicted.npy",y_outcome)

In [ ]:
y_outcome[0:300]

In [ ]:
file = np.load('predicted.npy')
file